In [1]:
# Place %autoreload 2 at the top of your notebook to ensure it applies to all subsequent imports.
%load_ext autoreload
%autoreload 2
 


In [2]:
from __future__ import absolute_import, division, print_function
import torch
from trainer_end_to_end import Trainer
from options import MonodepthOptions
import time
options = MonodepthOptions()
args = [
    '--data_path', '/mnt/nct-zfs/TCO-All/SharedDatasets/SCARED_Images_Resized/',
    # '--of_samples_num', '3', 
    '--batch_size', '10', 
    # '--enable_motion_computation', '', 
    # '--pose_model_type', 'separate_resnet', 
    # '--log_frequency', '2',
    # '--log_dir', '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/debug_reloc3r_backbone/relocxr',
    '--debug',
    ]
opts = options.parse_notebook(args)# trainer.set_eval()
# # opts
opts.dataset

2025-10-08 17:18:41.439755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759936721.453891  166909 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759936721.458052  166909 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759936721.469312  166909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759936721.469331  166909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759936721.469332  166909 computation_placer.cc:177] computation placer alr

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


'endovis'

In [3]:
# load model
trainer = Trainer(opts)
print('trainer initialized...')

DEBUG MODE
update options for debug purposes...
1234 is the seed!
learning rate: 0.0001
batch size is: 1
accumulate steps is: 1
effective batch size is: 1
Using DoMoRA
DARES_MoDoRA


/mnt/cluster/environments/jinjingxu/pkg/envs/cu12/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


DoRA params initialized!
DoRA rank is: [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
DAM mora rank is  [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
Vector MoRA params initialized!
MoDoRA in Depth model


/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/trainer_end_to_end.py:359: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.models["position_enc

loaded separate_resnet pose model...
CHECK: NO self.opt.predictive_mask
Model: depth_model
  Total params: 24981153 (23.82 MB)
  Total params trainable: 2933793 (2.80 MB)
Model: position_encoder
  Total params: 11698920 (11.16 MB)
  Total params trainable: 11698920 (11.16 MB)
Model: position
  Total params: 3154888 (3.01 MB)
  Total params trainable: 3154888 (3.01 MB)
Model: transform_encoder
  Total params: 11698920 (11.16 MB)
  Total params trainable: 11698920 (11.16 MB)
Model: transform
  Total params: 3157052 (3.01 MB)
  Total params trainable: 3157052 (3.01 MB)
Model: pose_encoder
  Total params: 11698920 (11.16 MB)
  Total params trainable: 11698920 (11.16 MB)
Model: pose
  Total params: 1314572 (1.25 MB)
  Total params trainable: 1314572 (1.25 MB)
Total params: 67704425 (64.57 MB)
Total params trainable: 45657065 (43.54 MB)
Training model named:
   mdp
Models and tensorboard events files are saved to:
   /home/jinjingxu/tmp
Training is using:
   cuda
use only of_samples_num samp

/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/trainer_end_to_end.py:617: UserWarning: Using StereoMIS dataset with split_appendix 
  warnings.warn(f"Using StereoMIS dataset with split_appendix {self.opt.split_appendix}")


Successfully loaded 6 trajectories
Loaded 6 trajectories
use only of_samples_num samples for OF, for OF training...
using 8 samples for OF, for speed up training..
Loading trajectories for 5 unique folders...
Successfully loaded 5 trajectories
Loaded 5 trajectories
Using dataset:
   StereoMIS
There are 8 training items and 8 validation items

trainer initialized...


/mnt/cluster/environments/jinjingxu/pkg/envs/cu12/lib/python3.11/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647348947/work/aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
# load trained model for test
import os
traiend_models_root = '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/unfzD_SCARED_b3accu1_posetr_net_reloc3r_9D_OPTTranse01_ROTAng01_Loss_regTrMagWt0_regRotAngMagWt0_386to386rather4096_depthScratchDAM_motionDftFlow_allRegLoss_doubleB_loss2weight100_tanhandclampSample_deltaMax001/0918-212235/models/weights_4'
if opts.pose_model_type == "separate_resnet":
    pose_encoder_path = os.path.join(traiend_models_root, "pose_encoder.pth")
    pose_decoder_path = os.path.join(traiend_models_root, "pose.pth")
    assert os.path.exists(pose_encoder_path), f"pose_encoder_path {pose_encoder_path} does not exist"
    assert os.path.exists(pose_decoder_path), f"pose_decoder_path {pose_decoder_path} does not exist"
    trainer.models["pose_encoder"].load_state_dict(torch.load(pose_encoder_path))
    print(f"loaded pose_encoder from {pose_encoder_path}")
    trainer.models["pose"].load_state_dict(torch.load(pose_decoder_path))
    print(f"loaded pose_decoder from {pose_decoder_path}")
    # opts.enable_motion_computation = True


    # load pretrained depth model
    depth_model_path = os.path.join(traiend_models_root, "depth_model.pth")
    depth_model_dict = torch.load(depth_model_path)
    trainer.models["depth_model"].load_state_dict(depth_model_dict)
    print(f"loaded depth_model from {depth_model_path}")

    # # load pretrained motion flow model
    # motion_flow_model_path = os.path.join(traiend_models_root, "motion_raft_flow.pth")
    # motion_flow_model_dict = torch.load(motion_flow_model_path)
    # print(trainer.models.keys())
    # # trainer.models["motion_raft_flow"].load_state_dict(motion_flow_model_dict)
    # # trainer.models["raft_flow"].load_state_dict(motion_flow_model_dict)
    # print(f"loaded motion_flow_model from {motion_flow_model_path}")


else:
    assert False, "pose_model_type should be separate_resnet or posetr_net"




/tmp/ipykernel_166909/2225298847.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trainer.models["pose_encoder"].load_state_dict(torch.load(pose_encoder_path))


loaded pose_encoder from /mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/unfzD_SCARED_b3accu1_posetr_net_reloc3r_9D_OPTTranse01_ROTAng01_Loss_regTrMagWt0_regRotAngMagWt0_386to386rather4096_depthScratchDAM_motionDftFlow_allRegLoss_doubleB_loss2weight100_tanhandclampSample_deltaMax001/0918-212235/models/weights_4/pose_encoder.pth
loaded pose_decoder from /mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/unfzD_SCARED_b3accu1_posetr_net_reloc3r_9D_OPTTranse01_ROTAng01_Loss_regTrMagWt0_regRotAngMagWt0_386to386rather4096_depthScratchDAM_motionDftFlow_allRegLoss_doubleB_loss2weight100_tanhandclampSample_deltaMax001/0918-212235/models/weights_4/pose.pth


/tmp/ipykernel_166909/2225298847.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trainer.models["pose"].load_state_dict(torch.load(pose_decoder_path))
/tmp/ipykernel_16

loaded depth_model from /mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/unfzD_SCARED_b3accu1_posetr_net_reloc3r_9D_OPTTranse01_ROTAng01_Loss_regTrMagWt0_regRotAngMagWt0_386to386rather4096_depthScratchDAM_motionDftFlow_allRegLoss_doubleB_loss2weight100_tanhandclampSample_deltaMax001/0918-212235/models/weights_4/depth_model.pth


In [5]:
# opts.pose_model_type
trainer.opt.pose_model_type,trainer.opt.unireloc3r_pose_estimation_mode,trainer.opt.enable_all_depth,trainer.opt.enable_motion_computation,trainer.opt.enable_mutual_motion

# trainer = Trainer(opts)
self = trainer #Trainer(opts)


# trainer.train()
"""Run the entire training pipeline
"""
self.epoch = 0
self.step = 0
self.start_time = time.time()


In [6]:
# # trainer = Trainer(opts)
# self = trainer #Trainer(opts)


# # trainer.train()
# """Run the entire training pipeline
# """
# self.epoch = 0
# # self.step = 0
# self.start_time = time.time()

#quick and convenient but confusing recontrooling!
# self.opt.min_depth = 10.0 #risky
# self.opt.min_depth = 0.1 #risky
# self.opt.max_depth = 150.0
# self.opt.gt_metric_rel_pose_as_estimates_debug = False

# trained fast3r
# depth: 0.2-0.3
# rel_pose_per_axis: 2e-4

with torch.no_grad():
    # for self.epoch in range(self.opt.num_epochs):
    # self.run_epoch(self.epoch)
    """Run a single epoch of training and validation
    """
    epoch = self.epoch
    print("Training")

    self.model_optimizer_0.zero_grad()# debug only: save_mem
    self.model_optimizer.zero_grad()# debug only: save_mem

    # for batch_idx, inputs in enumerate(self.train_loader):
    train_iter = iter(self.train_loader)
    inputs = next(train_iter)



    before_op_time = time.time()

    # position
    self.set_train_0()
    if self.opt.debug:
        self.freeze_params(keys = ['position_encoder'])#debug only to save_mem
    _, losses_0 = self.process_batch_0(inputs)
    self.model_optimizer_0.zero_grad()
    # losses_0["loss"].backward()#debug only
    torch.nn.utils.clip_grad_norm_(self.parameters_to_train_0, max_norm=1.0)
    self.model_optimizer_0.step()

    self.set_train()
    if self.opt.debug:
        self.freeze_params(keys = ['depth_model', 'pose', 'transform_encoder'])#debug only to save_mem
    outputs, losses= self.process_batch(inputs)
    self.model_optimizer.zero_grad()
    # losses["loss"].backward() #debug only
    torch.nn.utils.clip_grad_norm_(self.parameters_to_train, max_norm=1.0)
    self.model_optimizer.step()

    duration = time.time() - before_op_time

    # phase = batch_idx % self.opt.log_frequency == 0

    # # if phase:
    # # self.log_time(batch_idx, duration, losses["loss"].cpu().data)
    # self.log_time(0, duration, losses["loss"].cpu().data, losses_0["loss"].cpu().data, )
    # scalers_to_log = {
    #     "scalar/loss": losses["loss"],
    #     "scalar/loss_0": losses_0["loss"],
    #     # "scalar/trans_err": metric_errs["trans_err"].mean(),
    #     # "scalar/rot_err": metric_errs["rot_err"].mean()
    # }
    # for k, v in errs.items():
    #     print(f"scalar/{k}: {v}")
    #     scalers_to_log[f"scalar/{k}"] = v
    # concat_img, img_order_strs = self.log("train", inputs, outputs, scalers_to_log, compute_vis=True, online_vis=False)
    
    
    # Use the accumulated loss for logging (multiply back to show effective loss)
    # effective_loss = losses["loss"] * (self.opt.accumulate_steps / max(1, accumulate_step % self.opt.accumulate_steps))
    # self.log_time(batch_idx, duration, effective_loss.cpu().data)
    losses_to_log = {
        "loss": losses["loss"],
        "loss_0": losses_0["loss"],
    }
    errs_to_log = {}
    errs = self.compute_pose_metrics(inputs, outputs)
    for k, v in errs.items():
        assert len(v) == len(self.opt.frame_ids)-1, f'{k}: {v}'
        # v is a list of torch tensors, average over them
        errs_to_log[f"{k}"] = torch.mean(torch.stack(v)).item()

    self.log_time(self.step, duration, losses["loss"].cpu().data, losses_0["loss"].cpu().data, 
                    errs_to_log)
    scalers_to_log = {**losses_to_log, **errs_to_log}
    scalers_to_log = {f"scalar/{k}": v for k, v in scalers_to_log.items()}    
    # #compute_pose_err
    for k, v in errs.items():
        print(f"{k}: {v}")

    # print('trans_err:', outputs[("trans_err", 0)])
    # print('rot_err:', outputs[("rot_err", 0)])

 

    # concat_img, img_order_strs = self.log("train", inputs, outputs, 
    concat_img_list, img_order_strs_list = self.log("train", inputs, outputs, 
                scalers_to_log, 
                compute_vis=True)

    import os, cv2
    save_path = "/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/tmp"
    for concat_img, img_order_strs in zip(concat_img_list, img_order_strs_list):
    # concat_img, img_order_strs = self.log("train", inputs, outputs, 

        # remove the first char before the first _
        # img_order_strs = img_order_strs.split('_')[1:]
        # img_order_strs = '_'.join(img_order_strs)
        full_save_path = os.path.join(save_path, f"{img_order_strs}.png")
        cv2.imwrite(full_save_path, concat_img)
        print(f"saved train_concat_img.png in {full_save_path}")

    self.step += 1
    
    # break
        
    self.model_lr_scheduler.step()
    self.model_lr_scheduler_0.step()

    # if (self.epoch + 1) % self.opt.save_frequency == 0:
    #     self.save_model()
    # break
    


    # depth dim:
    # 256 320
    # 128,160,  
    # 64,80,
    # 32 40    # ===we go for this



# 0.06 0.04 0.04

Training


/mnt/cluster/environments/jinjingxu/pkg/envs/cu12/lib/python3.11/site-packages/torch/nn/functional.py:4902: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


////frame_id as src: -3
gt_tgt2src_rel_poses trans:
tensor([[ 0.0000e+00,  0.0000e+00, -9.3132e-10]], device='cuda:0')
pred_rel_poses_batch trans:
tensor([[ 1.3913e-04, -6.4708e-05, -3.7512e-04]], device='cuda:0')
////frame_id as src: 3
gt_tgt2src_rel_poses trans:
tensor([[-0.0353, -0.0319, -0.0004]], device='cuda:0')
pred_rel_poses_batch trans:
tensor([[ 1.1092e-04, -3.5445e-05,  1.2072e-04]], device='cuda:0')
epoch   0 | batch      0 | examples/s:   0.7 | loss: 0.02539 | loss_0: 0.01883 | time elapsed: 00h00m02s | time left: 00h00m00s
  Pose Errors: trans_err_ang_deg: 100.302 | trans_err_scale: 0.024 | rot_err_deg: 0.082 | pred_rel_trans_scale: 0.000 | pred_f0_depth_scale: 0.841 | pred_f0_cam_points_x_mean: 0.042 | pred_f0_cam_points_y_mean: -0.026 | pred_f0_cam_points_z_mean: 0.841 | 
trans_err_ang_deg: [tensor(89.9508, device='cuda:0'), tensor(110.6534, device='cuda:0')]
trans_err_scale: [tensor(0.0004, device='cuda:0'), tensor(0.0477, device='cuda:0')]
rot_err_deg: [tensor(0.0816,

[ WARN:0@18.176] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


In [7]:
# 97 -16 -43

In [8]:

# self.set_eval()
# try:
#     # inputs = self.val_iter.next()
#     inputs = next(self.val_iter)
# except StopIteration:
#     self.val_iter = iter(self.val_loader)
#     inputs = self.val_iter.next()

# with torch.no_grad():
#     # outputs, losses = self.process_batch_val(inputs)
#     """Pass a minibatch through the network and generate images and losses
#     """
#     for key, ipt in inputs.items():
#         inputs[key] = ipt.to(self.device)

#     if self.opt.pose_model_type == "shared":
#         # If we are using a shared encoder for both depth and pose (as advocated
#         # in monodepthv1), then all images are fed separately through the depth encoder.
#         print('CHECK: self.opt.pose_model_type == "shared"')
#         all_color_aug = torch.cat([inputs[("color_aug", i, 0)] for i in self.opt.frame_ids])
#         all_features = self.models["encoder"](all_color_aug)
#         all_features = [torch.split(f, self.opt.batch_size) for f in all_features]

#         features = {}
#         for i, k in enumerate(self.opt.frame_ids):
#             features[k] = [f[i] for f in all_features]

#         outputs = self.models["depth"](features[0])
#     else:
#         print('CHECK: self.opt.pose_model_type != "shared"')
#         # Otherwise, we only feed the image with frame_id 0 through the depth encoder
#         features = self.models["encoder"](inputs["color_aug", 0, 0])
#         outputs = self.models["depth"](features)

#     if self.opt.predictive_mask:
#         outputs["predictive_mask"] = self.models["predictive_mask"](features)

#     if self.use_pose_net:
#         outputs.update(self.predict_poses(inputs, features, outputs))

#     self.generate_images_pred(inputs, outputs)
#     losses = self.compute_losses_val(inputs, outputs)

#     # return outputs, losses

#     self.log("val", inputs, outputs, losses)
#     del inputs, outputs, losses